# 01. Basics #

In [1]:
#import the Quant Lib
import QuantLib as ql

# Let the today date whenwe want to value a instrument be
today = ql.Date(15,6,2020)

# we can set evaluationDate in QL as
ql.Settings.instance().evaluationDate = today
print(ql.Settings.instance().evaluationDate);
# prints..June 15th, 2020

# or you can do
today = ql.Date(15,12,2021);
ql.Settings.instance().setEvaluationDate(today)
print(ql.Settings.instance().evaluationDate)
# prints..December 15th, 2021

June 15th, 2020
December 15th, 2021


In [3]:
settlementDays = 2

# Holiday calendar of united states
calendar = ql.UnitedStates(ql.UnitedStates.FederalReserve)

forwardRate = 0.05

"""Day Counter provides methods for determining the length of a time period according to given market convention,
both as a number of days and as a year fraction."""
dayCounter = ql.Actual360()

# Construct flat forward rate term structure
flatForwardTermStructure = ql.FlatForward(settlementDays, calendar, forwardRate, dayCounter)

flatForwardTermStructure.referenceDate()

print("Max Date: ", flatForwardTermStructure.maxDate())

Max Date:  December 31st, 2199


In [4]:
today = ql.Date(15,6,2020)
ql.Settings.instance().evaluationDate = today

effectiveDate = ql.Date(15, 6, 2020)
terminationDate = ql.Date(15, 6, 2022)

In [5]:
schedule = ql.MakeSchedule(effectiveDate, terminationDate, ql.Period('6M'))

In [6]:
notional = [100.0]
rate = [0.05]
leg = ql.FixedRateLeg(schedule, dayCounter, notional, rate)

In [9]:
dayCounter = ql.Thirty360(ql.Thirty360.BondBasis)
rate = 0.03

"""
ql/Compounding.hpp
    //! Interest rate compounding rule
    enum Compounding { Simple = 0,          //!< \f$ 1+rt \f$
                       Compounded = 1,      //!< \f$ (1+r)^t \f$
                       Continuous = 2,      //!< \f$ e^{rt} \f$
                       SimpleThenCompounded, //!< Simple up to the first period then Compounded
                       CompoundedThenSimple //!< Compounded up to the first period then Simple
    };
"""

compoundingType = ql.Compounded

"""
ql/time/frequency.hpp
enum Frequency { NoFrequency = -1,     //!< null frequency
                     Once = 0,             //!< only once, e.g., a zero-coupon
                     Annual = 1,           //!< once a year
                     Semiannual = 2,       //!< twice a year
                     EveryFourthMonth = 3, //!< every fourth month
                     Quarterly = 4,        //!< every third month
                     Bimonthly = 6,        //!< every second month
                     Monthly = 12,         //!< once a month
                     EveryFourthWeek = 13, //!< every fourth week
                     Biweekly = 26,        //!< every second week
                     Weekly = 52,          //!< once a week
                     Daily = 365,          //!< once a day
                     OtherFrequency = 999  //!< some other unknown frequency
    };
"""

frequency = ql.Annual
interestRate = ql.InterestRate(rate, dayCounter, compoundingType, frequency)
print(interestRate)

3.000000 % 30/360 (Bond Basis) Annual compounding


In [11]:
ql.Settings.instance().evaluationDate = ql.Date(15,12,2020)
print(ql.CashFlows.npv(leg, interestRate, False))

7.376479977219242


## Array ##

## Matrix ##

In [15]:
print(ql.Matrix())
print(ql.Matrix(2,2))
print(ql.Matrix(2,2,0.5))


| 2.122e-314 9.97338e-312 |
| 4.42683e-321 6.95272e-310 |

| 0.5 0.5 |
| 0.5 0.5 |



In [14]:
A = ql.Matrix(3,3)
A[0][0]=0.2
A[0][1]=8.4
A[0][2]=1.5
A[1][0]=0.6
A[1][1]=1.4
A[1][2]=7.3
A[2][0]=0.8
A[2][1]=4.4
A[2][2]=3.2
print(A)

| 0.2 8.4 1.5 |
| 0.6 1.4 7.3 |
| 0.8 4.4 3.2 |



## Observable ##

In [16]:
flag = None
def raiseFlag():
    global flag
    flag = 1

me = ql.SimpleQuote(0.0)
obs = ql.Observer(raiseFlag)
obs.registerWith(me)
me.setValue(3.14)
if not flag:
    print("Case 1: Observer was not notified of market element change")
flag = None
obs.unregisterWith(me)
me.setValue(3.14)
if not flag:
    print("Case 2: Observer was not notified of market element change")

Case 2: Observer was not notified of market element change


## Quotes ##

In [17]:
s = ql.SimpleQuote(0.01)

In [18]:
s.value()
s.setValue(0.05)
s.isValid()

True

In [19]:
d1 = ql.SimpleQuote(0.06)
d2 = ql.DerivedQuote(ql.QuoteHandle(d1),lambda x: 10*x)

In [20]:
c1 = ql.SimpleQuote(0.02)
c2 = ql.SimpleQuote(0.03)

def f(x,y):
    return x+y

c3 = ql.CompositeQuote(ql.QuoteHandle(c1),ql.QuoteHandle(c2), f)
c3.value()

c4 = ql.CompositeQuote(ql.QuoteHandle(c1),ql.QuoteHandle(c2), lambda x,y:x+y)
c4.value()

0.05

In [21]:
deltaType = ql.DeltaVolQuote.Fwd    # Also supports: Spot, PaSpot, PaFwd
atmType = ql.DeltaVolQuote.AtmFwd   # Also supports: AtmSpot, AtmDeltaNeutral, AtmVegaMax, AtmGammaMax, AtmPutCall50

maturity = 1.0
volAtm, vol25DeltaCall, vol25DeltaPut = 0.08, 0.075, 0.095

atmDeltaQuote = ql.DeltaVolQuote(ql.QuoteHandle(ql.SimpleQuote(volAtm)), deltaType, maturity, atmType)
vol25DeltaPutQuote = ql.DeltaVolQuote(-0.25, ql.QuoteHandle(ql.SimpleQuote(vol25DeltaPut)), maturity, deltaType)
vol25DeltaCallQuote = ql.DeltaVolQuote(0.25, ql.QuoteHandle(ql.SimpleQuote(vol25DeltaCall)), maturity, deltaType)